In [1]:
# ch5/porto_seguro’s_safe_driver_prediction_baseline.ipynb

import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [2]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃 값 제거

In [3]:
all_features = all_data.columns # 전체 피처
all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

In [4]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [col for col in all_features if 'cat' in col] # 명목형 피처 추출

onehot_encoder = OneHotEncoder() # 원-핫 인코더 객체 생성
# 원-핫 인코딩 적용
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features]) 

encoded_cat_matrix

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

In [5]:
# 추가로 제거할 피처
drop_features = ['ps_ind_14', 'ps_ind_10_bin','ps_ind_11_bin', 
                 'ps_ind_12_bin','ps_ind_13_bin','ps_car_14']

# '1) 명목형 피처, 2) calc 태그가 포함된 피처, 3) 추가 제거할 피처'를 제외한 피처
remaining_features = [col for col in all_features if ('cat' not in col and \
                                                      'calc' not in col and \
                                                      col not in drop_features)]

In [6]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
                               encoded_cat_matrix],
                              format='csr')

In [7]:
num_train = train.shape[0] # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

In [8]:
def eval_gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]

    # sort rows on prediction column
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:, 0].argsort()][::-1, 0]
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]

    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples)

    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)

    # normalize to true Gini coefficient
    return G_pred * 1. / G_true

In [9]:
y_true = y_valid
y_pred = cv_train[valid_idx]

NameError: name 'y_valid' is not defined

In [10]:
# 실제 값과 예측 값의 크기가 같은지 확인
assert y_true.shape == y_pred.shape
# 데이터 개수
n_samples = y_true.shape[0]

# y_true 오름차순에 따라 y_true 값 재배열
true_order = y_true[y_true.argsort()]
# y_pred 오름차순에 따라 y_true 값 재배열
pred_order = y_true[y_pred.argsort()]

# 로렌츠 곡선
L_true = np.cumsum(true_order) / np.sum(true_order)
L_pred = np.cumsum(pred_order) / np.sum(pred_order)
L_mid = np.linspace(1 / n_samples, 1, n_samples)

# 예측이 완벽할 때 지니 계수
G_true = np.sum(L_mid - L_true)
# 예측 값에 대한 지니 계수
G_pred = np.sum(L_mid - L_pred)

# 정규화된 지니 계수
G_pred / G_true

NameError: name 'y_true' is not defined

0.29170498822890845

In [11]:
np.sum(true_order)

NameError: name 'true_order' is not defined

In [12]:
np.sum(pred_order)

NameError: name 'pred_order' is not defined

In [13]:
pred_order

NameError: name 'pred_order' is not defined

In [14]:
true_order

NameError: name 'true_order' is not defined

In [15]:
np.cumsum(pred_order)

NameError: name 'pred_order' is not defined

In [16]:
arr[arr[:, 0].argsort()]

NameError: name 'arr' is not defined

In [17]:
arr[arr[:, 1].argsort()]

NameError: name 'arr' is not defined

In [18]:
arr[arr[:, 1].argsort()][::-1, 0]

NameError: name 'arr' is not defined

In [19]:
arr[arr[:, 1].argsort()[::-1]][:, 0]

NameError: name 'arr' is not defined

In [20]:
y_true[:10]

NameError: name 'y_true' is not defined

In [21]:
y_pred[:10]

NameError: name 'y_pred' is not defined

In [22]:
arr = np.array([y_true, y_pred]).transpose()
arr

NameError: name 'y_true' is not defined

In [23]:
arr[:, 0].argsort()

NameError: name 'arr' is not defined

In [24]:
arr[np.array([0, 2, 1])]

NameError: name 'arr' is not defined

In [25]:
arr[arr[:, 0].argsort()][::-1, 0]

NameError: name 'arr' is not defined

In [26]:
arr[:, 1].argsort()

NameError: name 'arr' is not defined

In [27]:
arr[arr[:, 1].argsort()][::-1, 0]

NameError: name 'arr' is not defined

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [28]:
# 실제 값과 예측 값의 크기가 같은지 확인
assert y_true.shape == y_pred.shape
# 데이터 개수
n_samples = y_true.shape[0]
# 실제 값과 예측 값을 함께 배열로 저장
arr = np.array([y_true, y_pred]).transpose()

NameError: name 'y_true' is not defined

In [29]:
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True

In [30]:
import lightgbm as lgbm
from sklearn.model_selection import StratifiedKFold

save_cv = True
full_train = False
seed_rounds = 3

NFOLDS = 5
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)

num_boost_round = 10000

params = {"objective": "binary",
          "boosting_type": "gbdt",
          'metric': 'auc',
          "learning_rate": 0.01,
          "force_row_wise": True
          }

In [31]:
bayes_params = {
    'num_leaves':(25, 35), 
    'feature_fraction': (0.6, 0.9)
}

In [32]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# dtrain = lgbm.Dataset(X_train, y_train)
# dtest = lgbm.Dataset(X_test, y_test)

In [33]:
import lightgbm as lgbm

def lgb_eval(num_leaves, feature_fraction):
    
    params = {
        "n_estimator":200,
        'num_leaves':int(round(num_leaves)), 
        'feature_fraction': feature_fraction,
        'verbosity': -1
    }
    print("params:", params)    
    lgb_model = lgbm.train(params, dtrain,  
                       valid_sets=dtest, 
                       feval=gini, 
                       num_boost_round=1000,
                       verbose_eval=100, early_stopping_rounds=150)
    
    preds = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)
    
    gini_preds = eval_gini(y_test, preds)
    print('gini:', gini_preds)
    return gini_preds

In [34]:
# from bayes_opt import BayesianOptimization

# BO_lgb = BayesianOptimization(lgb_eval, bayes_params, random_state=0)

In [35]:
# BO_lgb.maximize(init_points=5, n_iter=10)

In [36]:
# BO_lgb.max

In [37]:
x_score = []

cv_train = np.zeros(X.shape[0])
cv_pred = np.zeros(X_test.shape[0])

best_iters = []
fold_scores = []

for train_idx, valid_idx in kfold.split(X, y):
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    dtrain = lgbm.Dataset(X_train, y_train)
    dvalid = lgbm.Dataset(X_valid, y_valid)

    lgb_model = lgbm.train(params, dtrain,  
                           valid_sets=dvalid, 
                           feval=gini, 
                           num_boost_round=1400,
                           verbose_eval=100, early_stopping_rounds=150)
    
    best_iters.append(lgb_model.best_iteration)
    cv_pred += lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)
    cv_train[valid_idx] += lgb_model.predict(X_valid)

    score = eval_gini(y_valid, cv_train[valid_idx])
    print(score)
    fold_scores.append(score)
    break

cv_pred /= NFOLDS

print("cv score:")
print(eval_gini(y, cv_train))
print("current score:", eval_gini(y, cv_train))
print(fold_scores)
print(best_iters, np.mean(best_iters))

x_score.append(eval_gini(y, cv_train))

[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1096
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 150 rounds
[100]	valid_0's auc: 0.631949	valid_0's gini: 0.263896
[200]	valid_0's auc: 0.638502	valid_0's gini: 0.277005
[300]	valid_0's auc: 0.641784	valid_0's gini: 0.283569
[400]	valid_0's auc: 0.643846	valid_0's gini: 0.287692
[500]	valid_0's auc: 0.645261	valid_0's gini: 0.290522
[600]	valid_0's auc: 0.64579	valid_0's gini: 0.291581
[700]	valid_0's auc: 0.645713	valid_0's gini: 0.291427
Early stopping, best iteration is:
[612]	valid_0's auc: 0.645852	valid_0's gini: 0.291705
0.29170498822890845
cv score:
0.009256676448323089
current score: 0.009256676448323089
[0.29170498822890845]
[612] 612.0


In [38]:
print(x_score)

[0.009256676448323089]


In [39]:
submission['target'] = cv_pred
submission.to_csv('submission.csv')